<a href="https://colab.research.google.com/github/ISaySalmonYouSayYes/LLM_mit_LangChain/blob/main/Langchain_cutomizeMemory_Summarizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Before you start**

1. Implementing summarizing before the system delete the oldest memory.
2. Use the token-Based memory. It outperformed the windows-Based one significantly.

### **Challenges - WindowsBase**
1. System would forget its rule soon. ----fixed
2. System knows it's talking to Kai, but it doesn't know I'm Kai.
3. Even if we asked system to summarize the details, it sometimes still just ignored it ----fixed, probably

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('openAI_Key')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai
    !pip install langgraph

# **1. Code**

## **1.1 Windows-based memory method**

In [31]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory, ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown
from langchain.chains import LLMChain


from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

MODEL = 'gpt-4o-mini'
TEMPLATE = """You are a helpful secretary. You can remember
        his personal information so that you can help him more. Format answers with markdown.

Current conversation:
{history}
Human: {input}
AI:"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)



class AutoSummarizingMemory(ConversationBufferWindowMemory):
    def __init__(self, k=5, **kwargs):
        super().__init__(k=k, **kwargs)

    def save_context(self, inputs, outputs):
        """
        Overrides the save_context method to automatically summarize the oldest messages
        if memory exceeds the `k` limit.
        """
        super().save_context(inputs, outputs)

        # Check if messages exceed the limit
        if len(self.chat_memory.messages) > self.k:
            self._summarize_oldest_messages()

    def _summarize_oldest_messages(self):
        """
        You are a helpful secretary helping your boss. You have to remember
        his personal information including numeral information so that you can help him more.
        Summarizes the oldest messages and replaces them with a single summary message.
        """
        messages = self.chat_memory.messages
        if len(messages) <= self.k:
            return  # No need to summarize if under the limit

        # Get the messages to summarize (all except the most recent `k` messages)
        oldest_messages = messages[:-self.k]

        # Format the history for summarization
        history = "\n".join(
            f"Human: {msg.content}" if isinstance(msg, HumanMessage) else f"AI: {msg.content}"
            for msg in oldest_messages
        )


        # Create a summarization prompt
        summary_prompt = f"Summarize the following conversation:\n\n{history}"
        llm = ChatOpenAI(model=MODEL, temperature=0.0)
        summary_response = llm.invoke([HumanMessage(content=summary_prompt)])
        summary = summary_response.content

        # Replace the oldest messages with a single summary message
        self.chat_memory.messages = [AIMessage(content=f"Summary: {summary}")] + messages[-self.k:]



def begin_conversation():
    # Initialize the OpenAI LLM
    llm = ChatOpenAI(
        model=MODEL,
        temperature=0.0,
        n=1
    )

    # Initialize memory with auto-summarization
    memory = AutoSummarizingMemory(k=5)  # Retains the last 5 exchanges after summarization
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation


def chat(conversation, prompt):
    print(f"Human: {prompt}")
    output = conversation.invoke(prompt)
    output = output['response']
    display_markdown(output, raw=True)

## **1.2 Token-Based memory method**

In [26]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory, ConversationTokenBufferMemory, ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown
from langchain.chains import LLMChain


from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

MODEL = 'gpt-4o-mini'
TEMPLATE = """You are a helpful secretary. You can remember
        his personal information so that you can help him more. Format answers with markdown.

Current conversation:
{history}
Human: {input}
AI:"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

def begin_conversation():
    # Initialize the OpenAI LLM
    llm = ChatOpenAI(
        model=MODEL,
        temperature=0.0,
        n=1
    )

    # Initialize memory with auto-summarization
    memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=20) #20 works very well
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation


def chat(conversation, prompt):
    print(f"Human: {prompt}")
    output = conversation.invoke(prompt)
    output = output['response']
    display_markdown(output, raw=True)

# **2. Result**

In [32]:
# Example usage
conversation = begin_conversation()

chat(conversation, "My name is Kai, generate a student number starting with S for me and remeber it")
chat(conversation, "My trip in Europe would end on the first of December")
chat(conversation, "My passport number is F987654321")
chat(conversation, "My birthday is 2000-01-01")
chat(conversation, "What is my student ID?")
chat(conversation, "When is my birthday?")
chat(conversation, "When is the end of my Journey?")
chat(conversation, "What is my passport number?")
chat(conversation, "Forget about my birthday")
chat(conversation, "When is my birthday?")
chat(conversation, "What is my name?")
chat(conversation, "When is my birthday?")
chat(conversation, "You don't have to forget my birthday anymore. When is my birthday?")
chat(conversation, "What is my student number?")
chat(conversation, "When is the end of my Journey?")
chat(conversation, "Who am I?")
chat(conversation, "Who is Kai?")
chat(conversation, "Who are you?")
chat(conversation, "What is my name?")
chat(conversation, "What is my student number?")

# Check the memory after conversation
print("Final memory:")
for message in conversation.memory.chat_memory.messages:
    print(f"{'Human' if isinstance(message, HumanMessage) else 'AI'}: {message.content}")


Human: My name is Kai, generate a student number starting with S for me and remeber it


Sure, Kai! Your student number is **S123456**. I'll remember it for you. If you need anything else, just let me know!

Human: My trip in Europe would end on the first of December


Got it, Kai! Your trip in Europe will end on **December 1st**. If there's anything else you'd like to share or need help with, just let me know!

Human: My passport number is F987654321


...

Human: What is my name?


I currently don't have your name stored. If you could provide it, I can remember it for future reference!

Human: What is my student number?


Your student number is **S123456**. If you need any further assistance or information, feel free to ask!

Final memory:
AI: Summary: In the conversation, Kai requests a student number, which the AI provides as **S123456**. Kai shares trip details, including an end date of **December 1st**, passport number **F987654321**, and birthday **January 1, 2000**. The AI confirms the student ID and birthday, then offers further assistance. Kai later asks the AI to confirm the end date, which the AI does. A human then asks for their passport number, and the AI responds with **F987654321**. The human requests to forget their birthday, and the AI confirms it has been forgotten. When the human asks when their birthday is, the AI reiterates that it has been forgotten. The conversation concludes with the human asking for their name, to which the AI responds that it does not have the name stored and requests it for future reference. The human then states they no longer want their birthday forgotten and asks when it is, but the AI maintains it has been forgotten. The human also inquires about Kai's identity

In [33]:
print(conversation.memory.buffer)

AI: Summary: In the conversation, Kai requests a student number, which the AI provides as **S123456**. Kai shares trip details, including an end date of **December 1st**, passport number **F987654321**, and birthday **January 1, 2000**. The AI confirms the student ID and birthday, then offers further assistance. Kai later asks the AI to confirm the end date, which the AI does. A human then asks for their passport number, and the AI responds with **F987654321**. The human requests to forget their birthday, and the AI confirms it has been forgotten. When the human asks when their birthday is, the AI reiterates that it has been forgotten. The conversation concludes with the human asking for their name, to which the AI responds that it does not have the name stored and requests it for future reference. The human then states they no longer want their birthday forgotten and asks when it is, but the AI maintains it has been forgotten. The human also inquires about Kai's identity, and the AI c